In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import pairwise_distances
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import ChebConv
from collections import Counter
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [2]:
'''读取数据'''
data_path = '../concatData.xlsx'
CN_MCI_Data = pd.read_excel(data_path,sheet_name="CN_MCI")

## 0、数据均衡

In [3]:
from sklearn.neighbors import NearestNeighbors

'''过采样方法'''
def find_and_remove_danger_samples(X, y, k=5): 
    X = np.array(X)
    y = np.array(y)
    majority_class = max(set(y), key=list(y).count)
    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(X)
    danger_indices = []
    for idx, x in enumerate(X):
        if y[idx] == majority_class:
            neighbors = neigh.kneighbors([x], return_distance=False)
            neighbor_labels = y[neighbors[0]]
            minority_count = np.sum(neighbor_labels != majority_class)
            if minority_count > k / 2:
                danger_indices.append(idx)
    X_cleaned = np.delete(X, danger_indices, axis=0)
    y_cleaned = np.delete(y, danger_indices, axis=0)
    
    return X_cleaned, y_cleaned

In [4]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import BorderlineSMOTE

cg_X = CN_MCI_Data[['PTGENDER', 'PTMARRY', 'AGE','PTEDUCAT', 
                  'cov_prs',
                  'cg17750572', 'cg01452847', 'cg13348062', 'cg18627235', 'cg15452204', 
                  'cg01821149', 'cg00884606', 'cg03354992', 'cg09173768', 'cg14515364',
                   'CDRSB', 'ADAS11', 'ADAS13', 'ADASQ4', 'MMSE',
                   'RAVLTimmediate', 'RAVLTlearning', 'RAVLTforgetting',
                   'RAVLTpercforgetting', 'LDELTOTAL', 'TRABSCOR', 'FAQ']]
cg_y = CN_MCI_Data['status']
print('原始 dataset shape %s' % Counter(cg_y))


'''过采样'''
sm = BorderlineSMOTE(
    random_state=42,
    kind="borderline-1",
    sampling_strategy={0: 250, 1: 345 },
    k_neighbors=5, #确定邻居点的数量
    m_neighbors=10) #指定在合成样本生成过程中从近邻点中选择多少个样本作为参考
X_bdsmote, y_bdsmote = sm.fit_resample(cg_X, cg_y)
print('过采样 dataset shape %s' % Counter(y_bdsmote))
# 合并为一个新的dataframe
y_bdsmote = pd.Series(y_bdsmote,name="status")
data_bdsmote = pd.concat([X_bdsmote, y_bdsmote], axis=1)


'''欠采样'''
X_cleaned, y_cleaned = find_and_remove_danger_samples(X_bdsmote.values, data_bdsmote["status"], k=100)
print(f"欠采样 dataset shape: {Counter(y_cleaned)}")
# 合并为一个新的dataframe
X_bdknn = pd.DataFrame(X_cleaned,columns=X_bdsmote.columns)
y_bdknn = pd.Series(y_cleaned,name="status")
data_bdknn = pd.concat([X_bdknn, y_bdknn], axis=1)

原始 dataset shape Counter({1: 345, 0: 209})
过采样 dataset shape Counter({1: 345, 0: 250})
欠采样 dataset shape: Counter({0: 250, 1: 240})


In [5]:
from sklearn.preprocessing import StandardScaler

# # 标准化
standard_scaler = StandardScaler()
standard_scaler.fit(X_bdknn)
X_bdknn = standard_scaler.transform(X_bdknn)
X_sample = pd.DataFrame(X_bdknn,columns=['PTGENDER', 'PTMARRY', 'AGE','PTEDUCAT','cov_prs',
                                          'cg17750572', 'cg01452847', 'cg13348062', 'cg18627235', 'cg15452204', 
                                          'cg01821149', 'cg00884606', 'cg03354992', 'cg09173768', 'cg14515364',
                                           'CDRSB', 'ADAS11', 'ADAS13', 'ADASQ4', 'MMSE',
                                           'RAVLTimmediate', 'RAVLTlearning', 'RAVLTforgetting',
                                           'RAVLTpercforgetting', 'LDELTOTAL', 'TRABSCOR', 'FAQ'])
X_data = X_sample[['PTGENDER', 'PTMARRY', 'AGE','PTEDUCAT','cov_prs',
                      'cg17750572', 'cg01452847', 'cg13348062', 'cg18627235', 'cg15452204', 
                      'cg01821149', 'cg00884606', 'cg03354992', 'cg09173768', 'cg14515364']]
X_cognitions = X_sample[['CDRSB', 'ADAS11', 'ADAS13', 'ADASQ4', 'MMSE',
                       'RAVLTimmediate', 'RAVLTlearning', 'RAVLTforgetting',
                       'RAVLTpercforgetting', 'LDELTOTAL', 'TRABSCOR', 'FAQ']]
y_data = y_bdknn

## 1、构建邻接矩阵

### （0）根据三类特征构建邻接矩阵

In [6]:
'''根据三类特征数据计算相邻矩阵'''

# 计算皮尔逊相关系数矩阵
corr = np.corrcoef(X_data, rowvar=True)
corr = corr.tolist()

# 构建 clinic_prs_methy_edge
matrix_clinic_prs_methy = np.zeros_like(corr)  # 初始化为零矩阵，形状与 corr 相同

for i in range(len(corr)):
    for j in range(len(corr[i])):
        if i != j:  # 排除对角线元素
            current_value = corr[i][j]
            if current_value > 0.5 or current_value <= -0.5:  # 只保留相似度大于 0.5 的样本对
                matrix_clinic_prs_methy[i][j] = 1  # 将满足条件的元素设为 1

### （1）权重矩阵

In [7]:
# 合并两种带边的节点矩阵
node_matrix  = matrix_clinic_prs_methy

'''根据12种认知评分数据计算相邻矩阵的权重（用于第二次卷积SAGE）'''
# 计算皮尔逊相关系数矩阵
cognitive_corr = np.corrcoef(X_cognitions, rowvar=True)
cognitive_corr = cognitive_corr.tolist()

# 根据认知评分增强的权重矩阵(将联合矩阵的权重赋为这个)
cog_weighted_matrix = node_matrix * cognitive_corr

### （3）数据处理

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# 归一化邻接矩阵
def normalize_adj(adj):
    adj = adj + np.eye(adj.shape[0])  # 增加自连接
    rowsum = np.array(adj.sum(1))
    d_inv = np.power(rowsum, -0.5).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat_inv = np.diag(d_inv)
    return d_mat_inv.dot(adj).dot(d_mat_inv)

In [9]:
X_features = np.array(X_data)
X_features = torch.from_numpy(X_features)
X_nodes = X_features.to(torch.float)

y_status = y_data.astype(int)
y_status = torch.from_numpy(y_status.values)
y_status = y_status.to(torch.long)

# 转换为PyTorch张量
cog_weighted_matrix = normalize_adj(cog_weighted_matrix)
cog_weighted_matrix = torch.tensor(cog_weighted_matrix, dtype=torch.float)

# 将邻接矩阵转换为稀疏表示
cog_edge_index = (cog_weighted_matrix > 0).nonzero(as_tuple=False).t()
cog_edge_weight = cog_weighted_matrix[cog_edge_index[0], cog_edge_index[1]]

# 创建PyTorch Geometric的数据对象
data = Data(x=X_nodes, y=y_status, edge_index=cog_edge_index, edge_attr=cog_edge_weight)

In [10]:
data

Data(x=[490, 15], edge_index=[2, 7348], edge_attr=[7348], y=[490])

## 2、GCN+SAGE 神经网络

In [11]:
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.data import Data
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix

class AD_GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(15, 8)  # 假设输入特征为15
        self.bn1 = nn.BatchNorm1d(8)
        self.conv2 = SAGEConv(8, 3)
        self.bn2 = nn.BatchNorm1d(3)
        self.fc = nn.Linear(3, 1)  # 二分类输出层
    
    def forward(self, x, edge_index):
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        return self.fc(x)  # 使用 BCEWithLogitsLoss，去掉 sigmoid



# 准备节点的索引进行交叉验证
node_indices = list(range(data.num_nodes))
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kfold.split(node_indices)):
    model = AD_GCN()  # 为每个折叠重新创建模型
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.BCEWithLogitsLoss()  # 使用 BCEWithLogitsLoss
    
    best_loss = float('inf')  # 初始化为正无穷大
    best_out = None  # 初始化为None
    best_metrics = {}

    for epoch in range(1, 201):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)  # 使用整个图
        loss = criterion(out[train_idx], data.y[train_idx].unsqueeze(1).float())
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            out = model(data.x, data.edge_index)
            pred = pred = (torch.sigmoid(out[test_idx]) > 0.5).long()  # 使用 threshold 转换为类别标签
            accuracy = accuracy_score(data.y[test_idx].cpu().numpy(), pred.cpu().numpy())
            precision = precision_score(data.y[test_idx].cpu(), pred.cpu(),average='weighted')
            mcc = matthews_corrcoef(data.y[test_idx].cpu(), pred.cpu())
 
        if loss.item() < best_loss:  # 更新最佳损失和指标
                best_loss = loss.item()
                best_metrics = {'MCC': mcc, 'Accuracy': accuracy, 'Precision': precision}
 
        # 打印最佳迭代结果
        if epoch % 200 == 0:  # 每20个epoch打印一次
            print(f'{best_metrics}')

{'MCC': 0.7335155457429191, 'Accuracy': 0.8673469387755102, 'Precision': 0.8677223397720292}
{'MCC': 0.6083449942418495, 'Accuracy': 0.7959183673469388, 'Precision': 0.813912661838929}
{'MCC': 0.7937710437710438, 'Accuracy': 0.8979591836734694, 'Precision': 0.8979591836734694}
{'MCC': 0.6954347826512214, 'Accuracy': 0.8469387755102041, 'Precision': 0.8486224489795917}
{'MCC': 0.7665135157089951, 'Accuracy': 0.8877551020408163, 'Precision': 0.8879585507220167}


In [37]:
# 初始化字典，每个指标都对应一个空列表
results = {
    "MCC": [],
    "Accuracy": [],
    "Precision": []
}

# 原始数据
CN_MCI_folds_data_weights = [
    {'MCC': 0.7335155457429191, 'Accuracy': 0.8673469387755102, 'Precision': 0.8677223397720292},
    {'MCC': 0.6083449942418495, 'Accuracy': 0.7959183673469388, 'Precision': 0.813912661838929},
    {'MCC': 0.7937710437710438, 'Accuracy': 0.8979591836734694, 'Precision': 0.8979591836734694},
    {'MCC': 0.6954347826512214, 'Accuracy': 0.8469387755102041, 'Precision': 0.8486224489795917},
    {'MCC': 0.7665135157089951, 'Accuracy': 0.8877551020408163, 'Precision': 0.8879585507220167}
]

# 遍历每一折的数据
for fold_data in CN_AD_folds_data_weights:
    results['MCC'].append(fold_data['MCC'])
    results['Accuracy'].append(fold_data['Accuracy'])
    results['Precision'].append(fold_data['Precision'])

# 输出结果，确保数据已正确添加
print(results)


# 邻接矩阵消融
# 'MCC': [0.8434032060389335, 0.8731534676229477, 0.9096846334879913, 0.8786764705882353, 0.819688599970537]

{'MCC': [0.8434032060389335, 0.8731534676229477, 0.9096846334879913, 0.8786764705882353, 0.819688599970537], 'Accuracy': [0.9253731343283582, 0.9393939393939394, 0.9545454545454546, 0.9393939393939394, 0.9090909090909091], 'Precision': [0.9274969028375907, 0.9421328671328671, 0.9594155844155844, 0.9393939393939394, 0.9105990783410137]}
